In [2]:
import json
import dotenv
import requests
import numpy as np
import pandas as pd
from time import sleep

In [3]:
def get_vacancies_id(header:dict, param:dict=None, URL:str='https://api.hh.ru/vacancies', per_page:int=100, time_dalay:float=0)->json:
    sleep(time_dalay)
    return requests.get(url=URL, headers=header, params=param).json()

In [4]:
JOB_TITLE = 'Аналитик данных'

USER_AGENT = 'https://my_own_test_app.com/page (followtotherabbit@gmail.com)'
BASE_URL='https://api.hh.ru/vacancies'

PER_PAGE = 100

header = {'User-Agent':USER_AGENT}
param = {'text': JOB_TITLE,
          'search_field': 'name',
          'page': 0,
          'per_page': PER_PAGE,
          'only_with_salary': True,
          'locale': 'RU'}

In [5]:
resp = get_vacancies_id(header, param, per_page=PER_PAGE)

VAC_CNT = resp['found']
PAGES_CNT = resp['pages']
current_page = resp['page']
VAC_CNT, PAGES_CNT, current_page

(233, 3, 0)

In [6]:
vac_ids = []
current_page = 0
PAGES_CNT = 999

while current_page <= PAGES_CNT-1:
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']
    PAGES_CNT = resp['pages']

    for item in resp['items']:
        vac_ids.append(item['id'])

    param['page'] = current_page + 1
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']


print("It's all OK" if len(set(vac_ids)) == VAC_CNT else f"Smt went WRONG\n{len(set(vac_ids))}___{VAC_CNT}")

It's all OK


In [26]:
for vac_id in vac_ids:
    vac_resp = get_vacancies_id(header, URL=BASE_URL+'/'+vac_id, time_dalay=0.5)
    break

vac_resp


{'id': '121218864',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Ведущий аналитик данных',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '47', 'name': 'Кемерово', 'url': 'https://api.hh.ru/areas/47'},
 'salary': {'from': 116000, 'to': 116000, 'currency': 'RUR', 'gross': False},
 'salary_range': {'from': 116000,
  'to': 116000,
  'currency': 'RUR',
  'gross': False,
  'mode': {'id': 'MONTH', 'name': 'За\xa0месяц'},
  'frequency': None},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Кемерово',
  'street': 'Карболитовская улица',
  'building': '1/4',
  'lat': 55.362644,
  'lng': 86.05714,
  'description': None,
  'raw': 'Кемерово, Карболитовская улица, 1/4',
  'metro': None,
  'metro_stations': []},
 'allow_messages': True,
 'experience': {'id': 'between1And3', 'name': 'От 1 года до 3 лет'},
 'schedule': {'id': 'fullDay', 'name': 'Полный день'},
 'employment': {'id': 'full', '

| Название поля   |      | Название поля в hh.api |                          |
| --------------- | ---- | ---------------------- | ------------------------ |
| vac_id          |      | id                     |                          |
| vac_name            |      | name                   |                          |
| city            |      | area_name              |                          |
| published_at    |      | published_at           | 2025-05-05T07:35:06+0300 |
| archived        | bool | archived               |                          |
| url             |      | alternate_url          |                          |
| emp_ed          |      | employer_id            |                          |
| emp_name        |      | employer_name          |                          |
| schedule_name   |      | schedule_name          | Полный день              |
| employment_name |      | employment_name        | Полная занятость         |

In [29]:
vacancies_df = pd.DataFrame(columns=['vac_id','vac_name','city','published_at','archived','alternate_url','employer_id','employer_name','schedule_name','employment_name'])


vacancies_df.loc[len(vacancies_df)] = [vac_resp['id'], \
                                                      vac_resp['name'], \
                                                      vac_resp['area']['name'], \
                                                      vac_resp['published_at'], \
                                                      vac_resp['archived'], \
                                                      vac_resp['alternate_url'], \
                                                      vac_resp['employer']['id'], \
                                                      vac_resp['employer']['name'], \
                                                      vac_resp['schedule']['name'], \
                                                      vac_resp['employment']['name']]


In [30]:
vacancies_df

,vac_id,vac_name,city,published_at,archived,alternate_url,employer_id,employer_name,schedule_name,employment_name
0,121218864,Ведущий аналитик данных,Кемерово,2025-06-02T12:33:29+0300,False,https://hh.ru/vacancy/121218864,9476,"Сибирский цемент, Холдинговая компания",Полный день,Полная занятость


In [ ]:
current_vac = res['items'][0]
fields = ['id',
          'name',
          {'area':['id',
                   'name',
                   'url']},
          'published_at',
          'archived',
          'alternate_url',
          {'employer':['id',
                       'name',
                       'accredited_it_employer',
                       'employer_rating',
                       'trusted']},
          'schedule',
          'employment']


for item in fields:
    if isinstance(item, dict):
        for k, v in item.items():
            try:
                for subitem in v:
                    print(f'{('_').join([k, subitem])} {current_vac[k][subitem]}')
            except:
                print(f'{('_').join([k, subitem])}, NOT FOUND')
    try:
        print(item,'\t\t\t\t',current_vac[item])
    except:
        print(f'not found {item}')
        continue